In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
data = pd.read_csv('./Prostate_Cancer.csv')

print(data.shape)

data.head()

(100, 10)


,id,diagnosis_result,radius,texture,perimeter,area,smoothness,compactness,symmetry,fractal_dimension
0,1,M,23,12,151,954,0.143,0.278,0.242,0.079
1,2,B,9,13,133,1326,0.143,0.079,0.181,0.057
2,3,M,21,27,130,1203,0.125,0.160,0.207,0.060
3,4,M,14,16,78,386,0.070,0.284,0.260,0.097
4,5,M,9,19,135,1297,0.141,0.133,0.181,0.059


In [3]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 100 non-null    int64  
 1   diagnosis_result   100 non-null    object 
 2   radius             100 non-null    int64  
 3   texture            100 non-null    int64  
 4   perimeter          100 non-null    int64  
 5   area               100 non-null    int64  
 6   smoothness         100 non-null    float64
 7   compactness        100 non-null    float64
 8   symmetry           100 non-null    float64
 9   fractal_dimension  100 non-null    float64
dtypes: float64(4), int64(5), object(1)
memory usage: 7.9+ KB


In [ ]:
# distribution of diagnosis results based on 

In [3]:
data.drop(columns=['id'], axis=1, inplace=True) # drop column 'id' from the dataset to avoid overfitting
data.head()

,diagnosis_result,radius,texture,perimeter,area,smoothness,compactness,symmetry,fractal_dimension
0,M,23,12,151,954,0.143,0.278,0.242,0.079
1,B,9,13,133,1326,0.143,0.079,0.181,0.057
2,M,21,27,130,1203,0.125,0.160,0.207,0.060
3,M,14,16,78,386,0.070,0.284,0.260,0.097
4,M,9,19,135,1297,0.141,0.133,0.181,0.059


In [3]:
# Importing the train_test_split function from sklearn
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=122)
print('Training data: ', train.shape)
print('Test data: ', test.shape)

Training data:  (80, 10)
Test data:  (20, 10)


In [4]:
# Splitting the data into train and test
Xtrain = train.drop(columns=['diagnosis_result'], axis=1) 
ytrain = train['diagnosis_result'] 

Xtest = test.drop(columns=['diagnosis_result'], axis=1)
ytest = test['diagnosis_result']

Best Parameters:  {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'splitter': 'best'}
Best Accuracy:  0.8375
Test Accuracy:  0.8


In [6]:
# 2nd: Random Forest Classification 
from sklearn.ensemble import RandomForestClassifier
# import GridSearchCV
from sklearn.model_selection import GridSearchCV

forest = RandomForestClassifier(random_state=122)

param_grid = {
    'n_estimators': [100, 200, 300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4],
    'max_features': ['sqrt', 'log2']
}

grid_search_forest = GridSearchCV(forest, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search_forest.fit(Xtrain, ytrain)

print('Best Parameters: ', grid_search_forest.best_params_)
print('Best Accuracy: ', grid_search_forest.best_score_)

best_forest = grid_search_forest.best_estimator_ # here best_forest is the best random forest model
print(best_forest)
# test accuracy
test_accuracy_forest = best_forest.score(Xtest, ytest) # score method calls the predict method and then compares the predicted values with the actual values
print('Test Accuracy: ', test_accuracy_forest)

Best Parameters:  {'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Best Accuracy:  0.8375
RandomForestClassifier(max_depth=10, min_samples_leaf=4, n_estimators=200,
                       random_state=122)
Test Accuracy:  0.9


In [10]:
# 3rd One: Naive Bayes Classification 
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
mNB = clf.fit(Xtrain,ytrain)
ypred = mNB.predict(Xtest)
print('Accuracy of Naive Bayes: ', mNB.score(Xtrain, ytrain))
test_accuracy = mNB.score(Xtest, ytest)
print('Test Accuracy: ', test_accuracy)

Accuracy of Naive Bayes:  0.7375
Test Accuracy:  0.85


In [11]:
# 4th Models - Logistic Regression
from sklearn.preprocessing import MinMaxScaler

# create the MinMaxScaler object
scaler = MinMaxScaler()
Xtrain_scaled = scaler.fit_transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

from sklearn.linear_model import LogisticRegression

# create a logistic regression Classifier
logreg = LogisticRegression(random_state=122, max_iter=5000)

param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

grid_search_logreg = GridSearchCV(logreg, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search_logreg.fit(Xtrain_scaled, ytrain)

print('Best Parameters: ', grid_search_logreg.best_params_)
print('Best Accuracy: ', grid_search_logreg.best_score_)

# get the best model
best_logreg = grid_search_logreg.best_estimator_

# evaluate the best model on the test set
test_accuracy = best_logreg.score(Xtest_scaled, ytest)
print('Test Accuracy: ', test_accuracy)

Best Parameters:  {'C': 1, 'penalty': 'l1', 'solver': 'saga'}
Best Accuracy:  0.85
Test Accuracy:  0.95


In [7]:
# 5th model: Support Vector Machine
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svc_model = SVC(random_state=122)

# Parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'sigmoid']
}
# 
grid_search_svc = GridSearchCV(svc_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search_svc.fit(Xtrain, ytrain)

# Access and print the results during grid search
results = grid_search_svc.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f'Mean accuracy for {params}: {mean_score:.3f}')

print('Best Parameters: ', grid_search_svc.best_params_)
print('Best Accuracy: ', grid_search_svc.best_score_)

best_svc = grid_search_svc.best_estimator_

test_accuracy = best_svc.score(Xtest, ytest)
print('Test Accuracy: ', test_accuracy)


exception calling callback for <Future at 0x16b0a73b190 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\joblib\externals\loky\_base.py", line 26, in _invoke_callbacks
    callback(self)
  File "C:\ProgramData\anaconda3\Lib\site-packages\joblib\parallel.py", line 385, in __call__
    self.parallel.dispatch_next()
  File "C:\ProgramData\anaconda3\Lib\site-packages\joblib\parallel.py", line 834, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\joblib\parallel.py", line 901, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\ProgramData\anaconda3\Lib\site-packages\joblib\parallel.py", line 819, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-pack

OSError: [Errno 22] Invalid argument

In [8]:
#  6th model: K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
# import GridSearchCV
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()

param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

grid_search_knn = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search_knn.fit(Xtrain, ytrain)

print('Best Parameters: ', grid_search_knn.best_params_)
print('Best Accuracy: ', grid_search_knn.best_score_)
best_knn = grid_search_knn.best_estimator_

test_accuracy = best_knn.score(Xtest, ytest)
print('Test Accuracy: ', test_accuracy)



Best Parameters:  {'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'uniform'}
Best Accuracy:  0.825
Test Accuracy:  0.85


In [10]:
#  import scaler
from sklearn.preprocessing import MinMaxScaler

# create the MinMaxScaler object
scaler = MinMaxScaler()

Xtrain_scaled = scaler.fit_transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

In [12]:
# Neural Network Model

from sklearn.neural_network import MLPClassifier
# import GridSearchCV
from sklearn.model_selection import GridSearchCV

mlp = MLPClassifier(random_state=122)

param_grid = {
    'hidden_layer_sizes': [(100,), (50, 50), (100, 100)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']
}

grid_search_mlp = GridSearchCV(mlp, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search_mlp.fit(Xtrain, ytrain)

print('Best Parameters: ', grid_search_mlp.best_params_)

print('Best Accuracy: ', grid_search_mlp.best_score_)

best_mlp = grid_search_mlp.best_estimator_

test_accuracy = best_mlp.score(Xtest, ytest)
print('Test Accuracy: ', test_accuracy)


Best Parameters:  {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (50, 50), 'learning_rate': 'constant', 'solver': 'sgd'}
Best Accuracy:  0.85
Test Accuracy:  0.8


In [12]:
# H1 : Hybrid of DT and RF
from sklearn.ensemble import VotingClassifier

voting_clf_DT_RF = VotingClassifier(estimators=[
    ('decision_tree', best_tree),   
    ('random_forest', best_forest) 
], voting='hard') 

voting_clf_DT_RF.fit(Xtrain, ytrain)

accuracy_DT_RF_test = voting_clf_DT_RF.score(Xtest, ytest)
accuracy_DT_RF_train = voting_clf_DT_RF.score(Xtrain, ytrain)
print('Hybrid of DT and RF Test Accuracy: ', accuracy_DT_RF_test)
print('Hybrid of DT and RF Train Accuracy: ', accuracy_DT_RF_train)


Hybrid of DT and RF Test Accuracy:  0.85
Hybrid of DT and RF Train Accuracy:  0.9125


In [13]:
# H2: Hybrid of LR and RF
voting_clf_LR_RF = VotingClassifier(estimators=[
    ('Logistic_reg', best_logreg),
    ('random_forest', best_forest) 
], voting='hard')  

voting_clf_LR_RF.fit(Xtrain, ytrain)

accuracy_LR_RF_test = voting_clf_LR_RF.score(Xtest, ytest)
accuracy_LR_RF_train = voting_clf_LR_RF.score(Xtrain, ytrain)
print('Hybrid of LR and RF Test Accuracy: ', accuracy_LR_RF_test)
print('Hybrid of LR and RF Train Accuracy: ', accuracy_LR_RF_train)

Hybrid of LR and RF Test Accuracy:  0.9
Hybrid of LR and RF Train Accuracy:  0.8625


In [32]:
# H3: Hybrid of LR and DT
voting_clf_LR_DT = VotingClassifier(estimators=[
    ('Logistic_reg', best_logreg),
    ('decision_tree', best_tree)
], voting='hard')  

voting_clf_LR_DT.fit(Xtrain, ytrain)

accuracy_LR_DT_test = voting_clf_LR_DT.score(Xtest, ytest)
accuracy_LR_DT_train = voting_clf_LR_DT.score(Xtrain, ytrain)
print('Hybrid of LR and DT Test Accuracy: ', accuracy_LR_DT_test)
print('Hybrid of LR and DT Train Accuracy: ', accuracy_LR_DT_train)

Hybrid of LR and DT Test Accuracy:  0.85
Hybrid of LR and DT Train Accuracy:  0.8625


In [29]:
# Hyperparameter Tuning for the Voting Classifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Define the estimators
estimators = [
    ('Logistic_reg', best_logreg),
    ('decision_tree', best_tree)
]

# Create the VotingClassifier
voting_clf_LR_DTH = VotingClassifier(estimators=estimators, voting='hard')

# Define the parameter grid for Logistic Regression
param_grid_logreg = {
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs', 'sag', 'saga']
}

# Define the parameter grid for Decision Tree
param_grid_tree = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the parameter grid for the VotingClassifier
param_grid_voting = {
    'weights': [[1,1], [1,2], [2,1], [2,2]]  # Different combinations of weights for LR and DT
}

# Create the GridSearchCV object
grid_search = GridSearchCV(voting_clf_LR_DTH, param_grid=param_grid_voting, cv=5, scoring='accuracy')

# Fit the GridSearchCV to the data
grid_search.fit(Xtrain, ytrain)

# Print the best parameters and best accuracy
print('Best Parameters: ', grid_search.best_params_)
print('Best Accuracy: ', grid_search.best_score_)

# Access the best estimator
best_voting_clf_LR_DT = grid_search.best_estimator_

# Calculate accuracy on test set
accuracy_LR_DT_test = best_voting_clf_LR_DT.score(Xtest, ytest)
print('Hybrid of LR and DT Test Accuracy: ', accuracy_LR_DT_test)


Best Parameters:  {'weights': [1, 2]}
Best Accuracy:  0.8375
Hybrid of LR and DT Test Accuracy:  0.8


In [33]:
# H4: Hybrid of RF and NB
voting_clf_RF_NB = VotingClassifier(estimators=[
    ('random_forest', best_forest),
    ('Naive_Bayes', mNB)
], voting='hard')

voting_clf_RF_NB.fit(Xtrain, ytrain)

accuracy_RF_NB_test = voting_clf_RF_NB.score(Xtest, ytest)
accuracy_RF_NB_train = voting_clf_RF_NB.score(Xtrain, ytrain)

print('Hybrid of RF and NB Test Accuracy: ', accuracy_RF_NB_test)
print('Hybrid of RF and NB Train Accuracy: ', accuracy_RF_NB_train)

Hybrid of RF and NB Test Accuracy:  0.85
Hybrid of RF and NB Train Accuracy:  0.7875


In [34]:
# H5: Hybrid of LR and DT and RF
voting_clf_LR_DT_RF = VotingClassifier(estimators=[
    ('Logistic_reg', best_logreg),
    ('decision_tree', best_tree),
    ('random_forest', best_forest)
], voting='hard')

voting_clf_LR_DT_RF.fit(Xtrain, ytrain) 

accuracy_LR_DT_RF_test = voting_clf_LR_DT_RF.score(Xtest, ytest) 
accuracy_LR_DT_RF_train = voting_clf_LR_DT_RF.score(Xtrain, ytrain)

print('Hybrid of LR and DT and RF Test Accuracy: ', accuracy_LR_DT_RF_test)
print('Hybrid of LR and DT and RF Train Accuracy: ', accuracy_LR_DT_RF_train)


Hybrid of LR and DT and RF Test Accuracy:  0.9
Hybrid of LR and DT and RF Train Accuracy:  0.9125


In [35]:
# Hyperparameter Tuning for the Voting Classifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Define the estimators
estimators = [
    ('Logistic_reg', best_logreg),
    ('decision_tree', best_tree),
    ('random_forest', best_forest)
]

# Create the VotingClassifier
voting_clf_LR_DT_RFH = VotingClassifier(estimators=estimators, voting='hard')

# Define the parameter grid for Logistic Regression
param_grid_logreg = {
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs', 'sag', 'saga']
}

# Define the parameter grid for Decision Tree
param_grid_tree = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the parameter grid for Random Forest
param_grid_forest = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the parameter grid for the VotingClassifier
param_grid_voting = {
    'weights': [[1,1,1], [1,2,1], [1,1,2], [2,1,1]]  # Different combinations of weights for LR, DT, and RF
}

# Create the GridSearchCV object
grid_search = GridSearchCV(voting_clf_LR_DT_RFH, param_grid=param_grid_voting, cv=5, scoring='accuracy')

# Fit the GridSearchCV to the data
grid_search.fit(Xtrain, ytrain)

# Print the best parameters and best accuracy
print('Best Parameters: ', grid_search.best_params_)
print('Best Accuracy: ', grid_search.best_score_)

# Access the best estimator
best_voting_clf_LR_DT_RF = grid_search.best_estimator_

# Calculate accuracy on test set
accuracy_LR_DT_RF_test = best_voting_clf_LR_DT_RF.score(Xtest, ytest)
print('Hybrid of LR, DT, and RF Test Accuracy: ', accuracy_LR_DT_RF_test)


Best Parameters:  {'weights': [1, 1, 2]}
Best Accuracy:  0.8375
Hybrid of LR, DT, and RF Test Accuracy:  0.9


In [42]:
# H : Hybrid of all 4
voting_clf_ALL = VotingClassifier(estimators=[
    ('Logistic_reg', best_logreg),
    ('decision_tree', best_tree),
    ('Naive Bayes', mNB),
    ('Random Forest', best_forest)
], voting='hard')  

voting_clf_ALL.fit(Xtrain, ytrain)

accuracy_ALL_test = voting_clf_ALL.score(Xtest, ytest)
accuracy_ALL_train = voting_clf_ALL.score(Xtrain, ytrain)
print('Hybrid of All ', accuracy_ALL_test)
print('Hybrid of all train ', accuracy_ALL_train)


Hybrid of All  0.9
Hybrid of all train  0.8625


In [43]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.ensemble import RandomForestClassifier

# Define the estimators
estimators = [
    ('Logistic_reg',  best_forest),
    ('decision_tree', best_forest),
    ('Naive Bayes', best_forest),
    ('Random Forest', best_forest)
]

# Create the VotingClassifier
voting_clf_ALLH = VotingClassifier(estimators=estimators, voting='hard')

# Define the parameter grid for Logistic Regression
param_grid_logreg = {
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs', 'sag', 'saga']
}

# Define the parameter grid for Decision Tree
param_grid_tree = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the parameter grid for Random Forest
param_grid_forest = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the parameter grid for the VotingClassifier
param_grid_voting = {
    'weights': [[1,1,1,1], [1,2,1,1], [1,1,2,1], [1,1,1,2], [2,1,1,1]]  # Different combinations of weights for LR, DT, NB, and RF
}

# Create the GridSearchCV object
grid_search = GridSearchCV(voting_clf_ALLH, param_grid=param_grid_voting, cv=5, scoring='accuracy')

# Fit the GridSearchCV to the data
grid_search.fit(Xtrain, ytrain)

# Print the best parameters and best accuracy
print('Best Parameters: ', grid_search.best_params_)
print('Best Accuracy: ', grid_search.best_score_)

# Access the best estimator
best_voting_clf_ALL = grid_search.best_estimator_

# Calculate accuracy on test set
accuracy_ALL_test = best_voting_clf_ALL.score(Xtest, ytest)
print('Hybrid of All Test Accuracy: ', accuracy_ALL_test)


Best Parameters:  {'weights': [1, 1, 1, 1]}
Best Accuracy:  0.8375
Hybrid of All Test Accuracy:  0.9


In [44]:
# comparision of the models
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, roc_auc_score

def performance_evaluation(X,y,clf,classifier_name=''):
    ypred = clf.predict(X)
    accuracy = clf.score(X,y)
    precision, recall, fscore, support = precision_recall_fscore_support(y, ypred, average='weighted', zero_division=0)
    metricName = ['Accuracy','Precision','Recall','F1_Score']
    metricValue = [accuracy,precision,recall,fscore]
    res = pd.DataFrame(metricValue, index=metricName, columns=[classifier_name])
    return res

In [45]:
# generate the performance summary for various models trained
tree_summary = performance_evaluation(Xtest,ytest,best_tree,'Decision Tree')
forest_summary = performance_evaluation(Xtest,ytest,best_forest,'Random Forest')
multinomialNB_summary = performance_evaluation(Xtest,ytest,mNB,'MultinomialNB')
logistic_summary = performance_evaluation(Xtest_scaled,ytest,best_logreg,'Logistic Regression')
hybrid_sum_DT_RF = performance_evaluation(Xtest,ytest,voting_clf_DT_RF,'DT_RF')
hybrid_sum_LR_RF = performance_evaluation(Xtest,ytest,voting_clf_LR_RF,'LR_RF')
hybrid_sum_LR_DT = performance_evaluation(Xtest,ytest,voting_clf_LR_DT,'LR_DT')
hybrid_sum_RF_NB = performance_evaluation(Xtest,ytest,voting_clf_RF_NB,'RF_NB')
hybrid_sum_LR_DT_RF = performance_evaluation(Xtest,ytest,voting_clf_LR_DT_RF,'LR_DT_RF')
hybrid_sum_ALL = performance_evaluation(Xtest,ytest,voting_clf_ALL,'ALL')

# combine the summary of each model into a dataframe 
comparison_df = pd.concat([tree_summary,forest_summary,multinomialNB_summary,logistic_summary,hybrid_sum_DT_RF,hybrid_sum_LR_RF,hybrid_sum_LR_DT,hybrid_sum_RF_NB,hybrid_sum_LR_DT_RF,hybrid_sum_ALL], axis=1)

# disply the summary dafaframe
display(comparison_df)

,Decision Tree,Random Forest,MultinomialNB,Logistic Regression,DT_RF,LR_RF,LR_DT,RF_NB,LR_DT_RF,ALL
Accuracy,0.8,0.9,0.850,0.950000,0.850000,0.9,0.850000,0.850,0.9,0.9
Precision,0.8,0.9,0.900,0.953333,0.860440,0.9,0.860440,0.900,0.9,0.9
Recall,0.8,0.9,0.850,0.950000,0.850000,0.9,0.850000,0.850,0.9,0.9
F1_Score,0.8,0.9,0.856,0.948589,0.852991,0.9,0.852991,0.856,0.9,0.9


In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Decision Tree model
tree = DecisionTreeClassifier(random_state=122)

# Define the hyperparameter grid
param_grid_tree = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4]
}

# GridSearchCV for Decision Tree
grid_search_tree = GridSearchCV(tree, param_grid_tree, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_tree.fit(Xtrain, ytrain)

# Print the best parameters and corresponding accuracy
print('Best Parameters (Decision Tree): ', grid_search_tree.best_params_)
print('Best Accuracy (Decision Tree): ', grid_search_tree.best_score_)

# Get the best Decision Tree model
best_tree = grid_search_tree.best_estimator_

# Random Forest model
forest = RandomForestClassifier(random_state=122)

# Define the hyperparameter grid
param_grid_forest = {
    'n_estimators': [100, 200, 300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4],
    'max_features': ['sqrt', 'log2']
}

# GridSearchCV for Random Forest
grid_search_forest = GridSearchCV(forest, param_grid_forest, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_forest.fit(Xtrain, ytrain)

# Print the best parameters and corresponding accuracy
print('Best Parameters (Random Forest): ', grid_search_forest.best_params_)
print('Best Accuracy (Random Forest): ', grid_search_forest.best_score_)

# Get the best Random Forest model
best_forest = grid_search_forest.best_estimator_

# Combine predictions using a majority voting scheme
hybrid_predictions = []
for i in range(len(Xtest)):
    tree_prediction = best_tree.predict(Xtest)[i]
    forest_prediction = best_forest.predict(Xtest)[i]

    predictions = [tree_prediction, forest_prediction]
    most_common_prediction = max(set(predictions), key=predictions.count)
    hybrid_predictions.append(most_common_prediction)

# Evaluate the accuracy of the hybrid model on the test set
hybrid_accuracy = accuracy_score(ytest, hybrid_predictions)
print("Hybrid Accuracy:", hybrid_accuracy)


Best Parameters (Decision Tree):  {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'splitter': 'best'}
Best Accuracy (Decision Tree):  0.8375
Best Parameters (Random Forest):  {'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Best Accuracy (Random Forest):  0.8375
Hybrid Accuracy: 0.85


In [56]:
# Hybrid of Decision Tree and Naive Bayes
from sklearn.ensemble import VotingClassifier

# Create a Voting Classifier
voting_clf = VotingClassifier(estimators=[
    # ('logistic_regression', best_logreg),   # best_tree is the best Decision Tree model
    # ('random_forest', best_forest),
    ('decision_tree', best_tree),
    ('naive_bayes', mNB)
], voting='hard')  # 'hard' voting means majority voting

# Fit the Voting Classifier to the data
voting_clf.fit(Xtrain, ytrain)

# Evaluate the accuracy of the Voting Classifier on the test set
accuracy_tree_nb = voting_clf.score(Xtest, ytest)
print('Voting Classifier Accuracy (Decision Tree and Naive Bayes): ', accuracy_tree_nb)

Voting Classifier Accuracy (Decision Tree and Naive Bayes):  0.8


In [57]:
# Hybrid of Logistic Regression and Random Forest
from sklearn.ensemble import VotingClassifier

# Create a Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('logistic_regression', best_logreg),   # best_tree is the best Decision Tree model
    ('random_forest', best_forest),
    # ('decision_tree', best_tree),
    # ('naive_bayes', mNB)
], voting='hard')  # 'hard' voting means majority voting

# Fit the Voting Classifier to the data
voting_clf.fit(Xtrain, ytrain)

# Evaluate the accuracy of the Voting Classifier on the test set
accuracy_logreg_forest = voting_clf.score(Xtest, ytest)
print('Voting Classifier Accuracy (Logistic Regression and Random Forest): ', accuracy_logreg_forest)

Voting Classifier Accuracy (Logistic Regression and Random Forest):  0.9


In [58]:
# Hybrid of Logistic Regression and Decision
from sklearn.ensemble import VotingClassifier

# Create a Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('logistic_regression', best_logreg),   # best_tree is the best Decision Tree model
    # ('random_forest', best_forest),
    ('decision_tree', best_tree),
    # ('naive_bayes', mNB)
], voting='hard')  # 'hard' voting means majority voting

# Fit the Voting Classifier to the data
voting_clf.fit(Xtrain, ytrain)

# Evaluate the accuracy of the Voting Classifier on the test set
accuracy_logreg_forest = voting_clf.score(Xtest, ytest)
print('Voting Classifier Accuracy (Logistic Regression and Decision Tree): ', accuracy_logreg_forest)

Voting Classifier Accuracy (Logistic Regression and Decision Tree):  0.85


In [59]:
# Hybrid of Logistic Regression and Naive Bayes
from sklearn.ensemble import VotingClassifier

# Create a Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('logistic_regression', best_logreg),   # best_tree is the best Decision Tree model
    # ('random_forest', best_forest),
    # ('decision_tree', best_tree),
    ('naive_bayes', mNB)
], voting='hard')  # 'hard' voting means majority voting

# Fit the Voting Classifier to the data
voting_clf.fit(Xtrain, ytrain)

# Evaluate the accuracy of the Voting Classifier on the test set
accuracy_logreg_mNB = voting_clf.score(Xtest, ytest)
print('Voting Classifier Accuracy (Logistic Regression and Naive Bayes): ', accuracy_logreg_mNB)

Voting Classifier Accuracy (Logistic Regression and Naive Bayes):  0.85


In [60]:
# Hybrid of Random Forest and Decision Tree and Logistic Regression
from sklearn.ensemble import VotingClassifier

# Create a Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('logistic_regression', best_logreg),   # best_tree is the best Decision Tree model
    ('random_forest', best_forest),
    ('decision_tree', best_tree),
    # ('naive_bayes', mNB)
], voting='hard')  # 'hard' voting means majority voting

# Fit the Voting Classifier to the data
voting_clf.fit(Xtrain, ytrain)

# Evaluate the accuracy of the Voting Classifier on the test set
accuracy_logreg_forest_tree = voting_clf.score(Xtest, ytest)
print('Voting Classifier Accuracy (Logistic Regression, Random Forest and Decision Tree): ', accuracy_logreg_forest_tree)

Voting Classifier Accuracy (Logistic Regression, Random Forest and Decision Tree):  0.9


In [61]:
# Hybrid of logistic regression, random forest, decision tree and naive bayes
from sklearn.ensemble import VotingClassifier

# Create a Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('logistic_regression', best_logreg),   # best_tree is the best Decision Tree model
    ('random_forest', best_forest),
    ('decision_tree', best_tree),
    ('naive_bayes', mNB)
], voting='hard')  # 'hard' voting means majority voting

# Fit the Voting Classifier to the data
voting_clf.fit(Xtrain, ytrain)

# Evaluate the accuracy of the Voting Classifier on the test set
accuracy_logreg_forest_tree_mNB = voting_clf.score(Xtest, ytest)
print('Voting Classifier Accuracy (Logistic Regression, Random Forest, Decision Tree and Naive Bayes): ', accuracy_logreg_forest_tree_mNB)

Voting Classifier Accuracy (Logistic Regression, Random Forest, Decision Tree and Naive Bayes):  0.9


In [63]:
# comparision of the models
# define a helper function to perform model evaluation based on key metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, roc_auc_score

def performance_evaluation(X,y,clf,classifier_name=''):
    ypred = clf.predict(X)
    accuracy = clf.score(X,y)
    precision, recall, fscore, support = precision_recall_fscore_support(y, ypred, average='weighted', zero_division=0)
    metricName = ['Accuracy','Precision','Recall','F1_Score']
    metricValue = [accuracy,precision,recall,fscore]
    res = pd.DataFrame(metricValue, index=metricName, columns=[classifier_name])
    return res